In [ ]:
using PMP, DataFrames, Distributions, Gadfly, Cairo, Fontconfig, Statistics, GLM, StatsBase

This notebook creates multiple samples out of different Pearson type I distributions then fits models to those samples. We use 4 distributions, of parameters
- A: a = 0, b = 50, $\alpha$ = 2, $\beta$ = 2;
- B: a = 0, b = 50, $\alpha$ = 1, $\beta$ = 3;
- C: a = 0, b = 50, $\alpha$ = 0.1, $\beta$ = 6;
- D: a = 0, b = 50, $\alpha$ = 0.1, $\beta$ = 15
For each of these distributions and for 6 different sizes (100, 500, 1000, 2000, 5000, 8000), we create 100 samples.

This code is not optimized. Particularly for the distributions C and D, the running time is very long.

# 4 parameters Pearson type I
---

# Distribution A
## Parameters a = 0, b = 50, $\alpha$ = 2, $\beta$ = 2
Creating samples

In [ ]:
d1 = PearsonType1(0, 50, 2, 2)
taille = [100, 500, 1000, 2000, 5000, 8000];

In [ ]:
nb = 100
sample_a = Array{Float64, 2}(undef, 100, taille[1])
sample_b = Array{Float64, 2}(undef, 100, taille[2])
sample_c = Array{Float64, 2}(undef, 100, taille[3])
sample_d = Array{Float64, 2}(undef, 100, taille[4])
sample_e = Array{Float64, 2}(undef, 100, taille[5])
sample_f = Array{Float64, 2}(undef, 100, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d1, taille[j])
    end
end

### Moments method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1, sample_names[j][i, :]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
end

df1 = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df1.aq2_5 = quantile_a[1,:]
df1.aq97_5 = quantile_a[2,:]

df1.bq2_5 = quantile_b[1,:]
df1.bq97_5 = quantile_b[2,:]

df1.αq2_5 = quantile_α[1,:]
df1.αq97_5 = quantile_α[2,:]

df1.βq2_5 = quantile_β[1,:]
df1.βq97_5 = quantile_β[2,:]

df1

In [ ]:
df_corr1 = append!(params_a, params_b)
append!(df_corr1, params_c)
append!(df_corr1, params_d)
append!(df_corr1, params_e)
append!(df_corr1, params_f);

In [ ]:
p1 = plot(df1, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
        style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=40, ymax=60), Guide.title("MM"), Guide.xlabel("Size")) 
p1

### Mixed method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = PMP.getinitialvalues(PearsonType1, sample_names[j][i, :])
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
    print(i)
end

df1m = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                 β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1m, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df1m.aq2_5 = quantile_a[1,:]
df1m.aq97_5 = quantile_a[2,:]

df1m.bq2_5 = quantile_b[1,:]
df1m.bq97_5 = quantile_b[2,:]

df1m.αq2_5 = quantile_α[1,:]
df1m.αq97_5 = quantile_α[2,:]

df1m.βq2_5 = quantile_β[1,:]
df1m.βq97_5 = quantile_β[2,:]

df1m

In [ ]:
df_corr1m = append!(params_a, params_b)
append!(df_corr1m, params_c)
append!(df_corr1m, params_d)
append!(df_corr1m, params_e)
append!(df_corr1m, params_f);

In [ ]:
p1m = plot(df1m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
        style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=40, ymax=60), Guide.title("Mixed"), 
    Guide.xlabel("Size"))
p1m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1, sample_names[j][i, :]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
    print(i)
end

df1mle = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                   β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1mle, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df1mle.aq2_5 = quantile_a[1,:]
df1mle.aq97_5 = quantile_a[2,:]

df1mle.bq2_5 = quantile_b[1,:]
df1mle.bq97_5 = quantile_b[2,:]

df1mle.αq2_5 = quantile_α[1,:]
df1mle.αq97_5 = quantile_α[2,:]

df1mle.βq2_5 = quantile_β[1,:]
df1mle.βq97_5 = quantile_β[2,:]

df1mle

In [ ]:
df_corr1mle = append!(params_a, params_b)
append!(df_corr1mle, params_c)
append!(df_corr1mle, params_d)
append!(df_corr1mle, params_e)
append!(df_corr1mle, params_f);

In [ ]:
p1mle = plot(df1mle, x=:taille, y=:b, Geom.line, Geom.point,
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=40, ymax=60), Guide.title("MLE"), 
    Guide.xlabel("Size"))
p1mle

In [ ]:
t1 = title(hstack(p1, p1m, p1mle), "Distribution A")

### Parameter correlation

In [ ]:
# moment method
lm1_aa = lm(@formula(α ~ a), df_corr1) # between a and α
lm1_bb = lm(@formula(β ~ b), df_corr1) # between b and β

# mixed method
lm1m_aa = lm(@formula(α ~ a), df_corr1m) # between a and α
lm1m_bb = lm(@formula(β ~ b), df_corr1m) # between b and β

# mle
lm1mle_aa = lm(@formula(α ~ a), df_corr1mle) # between a and α
lm1mle_bb = lm(@formula(β ~ b), df_corr1mle) # between b and β

In [ ]:
aα_distrA = plot(df_corr1, x=:a, y=:α, Geom.point, color=["MM"],
            layer(df_corr1m, x=:a, y=:α, Geom.point, color=["Mixed"]),
            layer(df_corr1mle, x=:a, y=:α, Geom.point, color=["MLE"]),
            intercept=[coef(lm1_aa)[1], coef(lm1m_aa)[1], coef(lm1mle_aa)[1]], 
            slope=[coef(lm1_aa)[2], coef(lm1m_aa)[2], coef(lm1mle_aa)[2]], 
            Geom.abline(color=["skyblue1", "yellow3", "hotpink"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
aα_distrA

In [ ]:
bβ_distrA = plot(df_corr1, x=:b, y=:β, Geom.point, color=["MM"],
            layer(df_corr1m, x=:b, y=:β, Geom.point, color=["Mixed"]),
            layer(df_corr1mle, x=:b, y=:β, Geom.point, color=["MLE"]),
            intercept=[coef(lm1_bb)[1], coef(lm1m_bb)[1], coef(lm1mle_bb)[1]], 
            slope=[coef(lm1_bb)[2], coef(lm1m_bb)[2], coef(lm1mle_bb)[2]], 
            Geom.abline(color=["skyblue1", "yellow3", "hotpink"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bβ_distrA


---
# Distribution B
## Parameters a = 0, b = 50, $\alpha$ = 1, $\beta$ = 3

Creating samples

In [ ]:
d2 = PearsonType1(0, 50, 1, 3)

sample_a = Array{Float64, 2}(undef, 200, taille[1])
sample_b = Array{Float64, 2}(undef, 200, taille[2])
sample_c = Array{Float64, 2}(undef, 200, taille[3])
sample_d = Array{Float64, 2}(undef, 200, taille[4])
sample_e = Array{Float64, 2}(undef, 200, taille[5])
sample_f = Array{Float64, 2}(undef, 200, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d2, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1, sample_names[j][i, :]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
end

df2 = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df2.aq2_5 = quantile_a[1,:]
df2.aq97_5 = quantile_a[2,:]

df2.bq2_5 = quantile_b[1,:]
df2.bq97_5 = quantile_b[2,:]

df2.αq2_5 = quantile_α[1,:]
df2.αq97_5 = quantile_α[2,:]

df2.βq2_5 = quantile_β[1,:]
df2.βq97_5 = quantile_β[2,:]

df2

In [ ]:
df_corr2 = append!(params_a, params_b)
append!(df_corr2, params_c)
append!(df_corr2, params_d)
append!(df_corr2, params_e)
append!(df_corr2, params_f); 

In [ ]:
p2 = plot(df2, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=30, ymax=90), Guide.title("MM"),
    Guide.xlabel("Size")) 
p2

### Mixed method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = PMP.getinitialvalues(PearsonType1, sample_names[j][i, :])
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
    print(i)
end

df2m = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                 β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2m, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df2m.aq2_5 = quantile_a[1,:]
df2m.aq97_5 = quantile_a[2,:]

df2m.bq2_5 = quantile_b[1,:]
df2m.bq97_5 = quantile_b[2,:]

df2m.αq2_5 = quantile_α[1,:]
df2m.αq97_5 = quantile_α[2,:]

df2m.βq2_5 = quantile_β[1,:]
df2m.βq97_5 = quantile_β[2,:]

df2m

In [ ]:
df_corr2m = append!(params_a, params_b)
append!(df_corr2m, params_c)
append!(df_corr2m, params_d)
append!(df_corr2m, params_e)
append!(df_corr2m, params_f); 

In [ ]:
p2m = plot(df2m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=30, ymax=90), Guide.title("Mixed"),
    Guide.xlabel("Size")) 
p2m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1, sample_names[j][i, :]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
    print(i)
end

df2mle = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                   β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2mle, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df2mle.aq2_5 = quantile_a[1,:]
df2mle.aq97_5 = quantile_a[2,:]

df2mle.bq2_5 = quantile_b[1,:]
df2mle.bq97_5 = quantile_b[2,:]

df2mle.αq2_5 = quantile_α[1,:]
df2mle.αq97_5 = quantile_α[2,:]

df2mle.βq2_5 = quantile_β[1,:]
df2mle.βq97_5 = quantile_β[2,:]

df2m

In [ ]:
df_corr2mle = append!(params_a, params_b)
append!(df_corr2mle, params_c)
append!(df_corr2mle, params_d)
append!(df_corr2mle, params_e)
append!(df_corr2mle, params_f); 

In [ ]:
p2mle = plot(df2mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=30, ymax=90), Guide.title("MLE"),
    Guide.xlabel("Size")) 
p2mle

In [ ]:
t2 = title(hstack(p2, p2m, p2mle), "Distribution B")

## Parameter correlation

In [ ]:
# moment method
lm2_aa = lm(@formula(α ~ a), df_corr2) # between a and α
lm2_bb = lm(@formula(β ~ b), df_corr2) # between b and β

# mixed method
lm2m_aa = lm(@formula(α ~ a), df_corr2m) # between a and α
lm2m_bb = lm(@formula(β ~ b), df_corr2m) # between b and β

# mle
lm2mle_aa = lm(@formula(α ~ a), df_corr2mle) # between a and α
lm2mle_bb = lm(@formula(β ~ b), df_corr2mle) # between b and β

In [ ]:
aα_distrB = plot(df_corr2, x=:a, y=:α, Geom.point, color=["MM"],
            layer(df_corr2m, x=:a, y=:α, Geom.point, color=["Mixed"]),
            layer(df_corr2mle, x=:a, y=:α, Geom.point, color=["MLE"]),
            intercept=[coef(lm2_aa)[1], coef(lm2m_aa)[1], coef(lm2mle_aa)[1]], 
            slope=[coef(lm2_aa)[2], coef(lm2m_aa)[2], coef(lm2mle_aa)[2]], 
            Geom.abline(color=["skyblue1", "yellow3", "hotpink"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
aα_distrB

In [ ]:
bβ_distrB = plot(df_corr2, x=:b, y=:β, Geom.point, color=["MM"],
            layer(df_corr2m, x=:b, y=:β, Geom.point, color=["Mixed"]),
            layer(df_corr2mle, x=:b, y=:β, Geom.point, color=["MLE"]),
            intercept=[coef(lm2_bb)[1], coef(lm2m_bb)[1], coef(lm2mle_bb)[1]], 
            slope=[coef(lm2_bb)[2], coef(lm2m_bb)[2], coef(lm2mle_bb)[2]], 
            Geom.abline(color=["skyblue1", "yellow3", "hotpink"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bβ_distrB


---
# Distribution C
## Parameters a = 0, b = 50, $\alpha$ = 0.1, $\beta$ = 6

Creating samples

In [ ]:
nb = 100
d3 = PearsonType1(0, 50, 0.1, 6)

sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d3, taille[j])
    end
end 

### Moment method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

nothing_compteur = 0

for i in 1:nb
    for j in 1:length(taille)
        fit = fit_mme(PearsonType1, sample_names[j][i, :])
        if fit==nothing
            nothing_compteur = nothing_compteur + 1
        else
            fit = params(fit)
            estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
            append!(names[j], estimation)
        end
    end
end

df3 = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df3.aq2_5 = quantile_a[1,:]
df3.aq97_5 = quantile_a[2,:]

df3.bq2_5 = quantile_b[1,:]
df3.bq97_5 = quantile_b[2,:]

df3.αq2_5 = quantile_α[1,:]
df3.αq97_5 = quantile_α[2,:]

df3.βq2_5 = quantile_β[1,:]
df3.βq97_5 = quantile_β[2,:]

df3

In [ ]:
df_corr3 = append!(params_a, params_b)
append!(df_corr3, params_c)
append!(df_corr3, params_d)
append!(df_corr3, params_e)
append!(df_corr3, params_f); 

In [ ]:
nothing_compteur/nb

In [ ]:
p3 = plot(df3, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=0, ymax=240), Guide.title("MM"),
    Guide.xlabel("Size")) 
p3

### Mixed method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

nothing_compteur = 0

for i in 1:nb
    for j in 1:length(taille)
        test = fit_mme(PearsonType1, sample_names[j][i, :])
        if test==nothing
            nothing_compteur = nothing_compteur + 1
        else
            fit = PMP.getinitialvalues(PearsonType1, sample_names[j][i, :])
            estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
            append!(names[j], estimation)
        end
    end
    print(i)
end

df3m = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                 β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3m, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df3m.aq2_5 = quantile_a[1,:]
df3m.aq97_5 = quantile_a[2,:]

df3m.bq2_5 = quantile_b[1,:]
df3m.bq97_5 = quantile_b[2,:]

df3m.αq2_5 = quantile_α[1,:]
df3m.αq97_5 = quantile_α[2,:]

df3m.βq2_5 = quantile_β[1,:]
df3m.βq97_5 = quantile_β[2,:]

df3m

In [ ]:
df_corr3m = append!(params_a, params_b)
append!(df_corr3m, params_c)
append!(df_corr3m, params_d)
append!(df_corr3m, params_e)
append!(df_corr3m, params_f); 

In [ ]:
nothing_compteur/nb

In [ ]:
p3m = plot(df3m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt),
            Coord.cartesian(ymin=0, ymax=60), Guide.title("Mixed"),
    Guide.xlabel("Size")) 
p3m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1, sample_names[j][i, :]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimation)
    end
    print(i)
end

df3mle = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                   β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3mle, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df3mle.aq2_5 = quantile_a[1,:]
df3mle.aq97_5 = quantile_a[2,:]

df3mle.bq2_5 = quantile_b[1,:]
df3mle.bq97_5 = quantile_b[2,:]

df3mle.αq2_5 = quantile_α[1,:]
df3mle.αq97_5 = quantile_α[2,:]

df3mle.βq2_5 = quantile_β[1,:]
df3mle.βq97_5 = quantile_β[2,:]

df3mle

In [ ]:
df_corr3mle = append!(params_a, params_b)
append!(df_corr3mle, params_c)
append!(df_corr3mle, params_d)
append!(df_corr3mle, params_e)
append!(df_corr3mle, params_f); 

In [ ]:
p3mle = plot(df3mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt), Guide.title("MLE"),
    Guide.xlabel("Size")) 
p3mle

In [ ]:
t3 = title(hstack(p3, p3m, p3mle), "Distribution C")

## Parameter correlation

In [ ]:
# moment method
lm3_aa = lm(@formula(α ~ a), df_corr3) # between a and α
lm3_bb = lm(@formula(β ~ b), df_corr3) # between b and β

# mixed method
lm3m_aa = lm(@formula(α ~ a), df_corr3m) # between a and α
lm3m_bb = lm(@formula(β ~ b), df_corr3m) # between b and β

# mle
lm3mle_aa = lm(@formula(α ~ a), df_corr3mle) # between a and α
lm3mle_bb = lm(@formula(β ~ b), df_corr3mle) # between b and β

In [ ]:
# MM
aα_distrC = plot(df_corr3, x=:a, y=:α, Geom.point,
            intercept=[coef(lm3_aa)[1]], slope=[coef(lm3_aa)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

bβ_distrC = plot(df_corr3, x=:b, y=:β, Geom.point,
            intercept=[coef(lm3_bb)[1]], slope=[coef(lm3_bb)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

corr_distrC = title(hstack(aα_distrC, bβ_distrC), "Moment method")

In [ ]:
# Mixed
aαm_distrCm = plot(df_corr3m, x=:a, y=:α, Geom.point,
            intercept=[coef(lm3m_aa)[1]], slope=[coef(lm3m_aa)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

bβm_distrCm = plot(df_corr3m, x=:b, y=:β, Geom.point,
            intercept=[coef(lm3m_bb)[1]], slope=[coef(lm3m_bb)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

corr_distrCm = title(hstack(aαm_distrC, bβm_distrC), "Mixed method")

In [ ]:
# MLE 
aαmle_distrC = plot(df_corr3mle, x=:a, y=:α, Geom.point,
            intercept=[coef(lm3mle_aa)[1]], slope=[coef(lm3mle_aa)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

bβmle_distrC = plot(df_corr3mle, x=:b, y=:β, Geom.point,
            intercept=[coef(lm3mle_bb)[1]], slope=[coef(lm3mle_bb)[2]], Geom.abline(color=["red"], style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

corr_distrCmle = title(hstack(aαmle_distrC, bβmle_distrC), "Maximum likelihood estimator")            

---
# Distribution D
## Parameters : a = 0 b = 50, $\alpha$ = 0.1, $\beta$ = 15
Creating samples

In [ ]:
nb = 100
d4 = PearsonType1(0, 50, 0.1, 15)

sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d4, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

nothing_compteur = 0

for i in 1:nb
    for j in 1:length(taille)
        fit = fit_mme(PearsonType1, sample_names[j][i, :])
        if fit==nothing
            nothing_compteur = nothing_compteur + 1
        else
            fit = params(fit)
            estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
            append!(names[j], estimation)
        end
    end
end

df4 = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df4.aq2_5 = quantile_a[1,:]
df4.aq97_5 = quantile_a[2,:]

df4.bq2_5 = quantile_b[1,:]
df4.bq97_5 = quantile_b[2,:]

df4.αq2_5 = quantile_α[1,:]
df4.αq97_5 = quantile_α[2,:]

df4.βq2_5 = quantile_β[1,:]
df4.βq97_5 = quantile_β[2,:]

df4

In [ ]:
df_corr4 = append!(params_a, params_b)
append!(df_corr4, params_c)
append!(df_corr4, params_d)
append!(df_corr4, params_e)
append!(df_corr4, params_f); 

In [ ]:
p4 = plot(df4, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt), Guide.title("MM"), 
    Guide.xlabel("Size"))
p4

In [ ]:
nothing_compteur/nb

### Mixed method

In [ ]:
# for next mle, we will keep the fits estimated here
iparams_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
iparams_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
iparams_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
iparams_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
iparams_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
iparams_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
inames = [iparams_a, iparams_b, iparams_c, iparams_d, iparams_e, iparams_f]

nothing_compteur = 0

for i in 1:nb
    for j in 1:length(taille)
        test = fit_mme(PearsonType1, sample_names[j][i, :])
        if test==nothing
            nothing_compteur = nothing_compteur + 1
        else
            fit = PMP.getinitialvalues(PearsonType1, sample_names[j][i, :])
            estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
            append!(inames[j], estimation)
        end
    end
    print(i)
end

df4m = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                 β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4m, DataFrame(taille=taille[j], a=mean(inames[j].a), b=mean(inames[j].b),  α=mean(inames[j].α), 
            β=mean(inames[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(inames[j].a, [.025, .975])
    quantile_b[:,j] = quantile(inames[j].b, [.025, .975])
    quantile_α[:,j] = quantile(inames[j].α, [.025, .975])
    quantile_β[:,j] = quantile(inames[j].β, [.025, .975])
end

df4m.aq2_5 = quantile_a[1,:]
df4m.aq97_5 = quantile_a[2,:]

df4m.bq2_5 = quantile_b[1,:]
df4m.bq97_5 = quantile_b[2,:]

df4m.αq2_5 = quantile_α[1,:]
df4m.αq97_5 = quantile_α[2,:]

df4m.βq2_5 = quantile_β[1,:]
df4m.βq97_5 = quantile_β[2,:]

df4m

In [ ]:
df_corr4m = append!(iparams_a, iparams_b)
append!(df_corr4m, iparams_c)
append!(df_corr4m, iparams_d)
append!(df_corr4m, iparams_e)
append!(df_corr4m, iparams_f); 

In [ ]:
nothing_compteur/nb

In [ ]:
p4m = plot(df4m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt), Guide.title("Mixed"), 
    Guide.xlabel("Size"))
p4m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(a = Float64[], b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1, sample_names[j][i, :], 
                             [inames[j][i,1], inames[j][i,2], inames[j][i,3], inames[j][i,4]]))
        estimation = DataFrame(a=fit[1], b=fit[2], α=fit[3], β=fit[4])
        append!(names[j], estimatißon)
    end
    print(i)
end

In [ ]:
df4mle = DataFrame(taille = taille[1], a = mean(params_a.a), b = mean(params_a.b), α = mean(params_a.α), 
                   β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4mle, DataFrame(taille=taille[j], a=mean(names[j].a), b=mean(names[j].b),  α=mean(names[j].α), 
            β=mean(names[j].β)))
end

quantile_a = Array{Float64, 2}(undef, 2, 6)
quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_a[:,j] = quantile(names[j].a, [.025, .975])
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df4mle.aq2_5 = quantile_a[1,:]
df4mle.aq97_5 = quantile_a[2,:]

df4mle.bq2_5 = quantile_b[1,:]
df4mle.bq97_5 = quantile_b[2,:]

df4mle.αq2_5 = quantile_α[1,:]
df4mle.αq97_5 = quantile_α[2,:]

df4mle.βq2_5 = quantile_β[1,:]
df4mle.βq97_5 = quantile_β[2,:]

df4mle

In [ ]:
df_corr4mle = append!(params_a, params_b)
append!(df_corr4mle, params_c)
append!(df_corr4mle, params_d)
append!(df_corr4mle, params_e)
append!(df_corr4mle, params_f); 

In [ ]:
p4mle = plot(df4mle, x=:taille, y=:b, Geom.point, Geom.line,
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    style(major_label_font_size=16pt, minor_label_font_size=14pt), Guide.title("MLE"), 
    Guide.xlabel("Size"))
p4mle

In [ ]:
t4 = title(hstack(p4, p4m, p4mle), "Distribution D")

In [ ]:
# moment method
lm4_aa = lm(@formula(α ~ a), df_corr4) # between a and α
lm4_bb = lm(@formula(β ~ b), df_corr4) # between b and β

# mixed method
lm4m_aa = lm(@formula(α ~ a), df_corr4m) # between a and α
lm4m_bb = lm(@formula(β ~ b), df_corr4m) # between b and β

# mle
lm4mle_aa = lm(@formula(α ~ a), df_corr4mle) # between a and α
lm4mle_bb = lm(@formula(β ~ b), df_corr4mle) # between b and β

In [ ]:
# MM
aα_distrD = plot(df_corr4, x=:a, y=:α, Geom.point,
            intercept=[coef(lm4_aa)[1]], slope=[coef(lm4_aa)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

bβ_distrD = plot(df_corr4, x=:b, y=:β, Geom.point,
            intercept=[coef(lm4_bb)[1]], slope=[coef(lm4_bb)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
        
corr_distrD = title(hstack(aα_distrD, bβ_distrD), "Moment method")

In [ ]:
# Mixed
aαm_distrD = plot(df_corr4m, x=:a, y=:α, Geom.point,
            intercept=[coef(lm4m_aa)[1]], slope=[coef(lm4m_aa)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

bβm_distrD = plot(df_corr4m, x=:b, y=:β, Geom.point,
            intercept=[coef(lm4m_bb)[1]], slope=[coef(lm4m_bb)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

corr_distrDm = title(hstack(aαm_distrD, bβm_distrD), "Mixed method")

In [ ]:
# MLE
aαmle_distrD = plot(df_corr4mle, x=:a, y=:α, Geom.point,
            intercept=[coef(lm4mle_aa)[1]], slope=[coef(lm4mle_aa)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt)) 

bβmle_distrD = plot(df_corr4mle, x=:b, y=:β, Geom.point,
            intercept=[coef(lm4mle_bb)[1]], slope=[coef(lm4mle_bb)[2]], Geom.abline(color="red", style=:dash),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

corr_distrDmle = title(hstack(aαmle_distrD, bβmle_distrD), "Maximum likelihood estimation")